In [2]:
import pandas as pd
from pathlib import Path

In [3]:
BASE_DIR = Path().resolve().parent

fossil_raw_file_path = BASE_DIR / "data" / "raw" / "Fossil CO2 emissions.csv"
source_path  = BASE_DIR / "data" / "raw" / "National_Fossil_Carbon_Emissions_2024v1.0.xlsx"
landuse_file_path  = BASE_DIR / "data" / "raw" / "National_LandUseChange_Carbon_Emissions_2024v1.0.xlsx"

OUTPUT_PATH = BASE_DIR / "data" / "processed" / "Final_CO2_Emissions_Dataset.csv"

In [4]:
# Step 1: Load raw fossil CO2 dataset
df_fossil_raw = pd.read_csv(fossil_raw_file_path)

# Step 2: Define renaming dictionary for harmonizing country names
rename_dict = {
    "TÜRKIYE": "TURKEY",
    "CÔTE D'IVOIRE": "COTE D'IVOIRE",
    "BONAIRE, SAINT EUSTATIUS AND SABA": "BONAIRE SINT EUSTATIUS AND SABA",
    "BRUNEI DARUSSALAM": "BRUNEI",
    "CURAÇAO": "CURACAO",
    "DEMOCRATIC REPUBLIC OF THE CONGO": "DEMOCRATIC REPUBLIC OF CONGO",
    "FAEROE ISLANDS": "FAROE ISLANDS",
    "GLOBAL": "WORLD",
    "MICRONESIA (FEDERATED STATES OF)": "MICRONESIA (COUNTRY)",
    "STATE OF PALESTINE": "PALESTINE",
    "TIMOR-LESTE": "EAST TIMOR",
    "USA": "UNITED STATES",
    "VIET NAM": "VIETNAM",
    "WALLIS AND FUTUNA ISLANDS": "WALLIS AND FUTUNA",
    "KUWAITI OIL FIRES": "KUWAITI OIL FIRES"
}

# Step 3: Clean and rename relevant columns
df_fossil_cleaned = pd.DataFrame({
    "Country": df_fossil_raw["Country"].str.upper().replace(rename_dict),
    "Country_Abbreviation": df_fossil_raw["ISO 3166-1 alpha-3"],
    "Year": df_fossil_raw["Year"],
    "Total": df_fossil_raw["Total"],
    "Coal": df_fossil_raw["Coal"],
    "Oil": df_fossil_raw["Oil"],
    "Gas": df_fossil_raw["Gas"],
    "Cement": df_fossil_raw["Cement"],
    "Flaring": df_fossil_raw["Flaring"],
    "Other": df_fossil_raw["Other"]
})

# Step 4: Normalize whitespace and trim country names
df_fossil_cleaned["Country"] = (
    df_fossil_cleaned["Country"]
    .astype(str)
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
)

# Step 5: Special handling for 'PALAU' historical anomaly
mask_palau_pre1992 = (
    (df_fossil_cleaned["Country"] == "PACIFIC ISLANDS (PALAU)") & 
    (df_fossil_cleaned["Year"] < 1992)
)
mask_palau_post1992 = (
    (df_fossil_cleaned["Country"] == "PALAU") & 
    (df_fossil_cleaned["Year"] >= 1992)
)

df_fossil_palau_fixed = df_fossil_cleaned[mask_palau_pre1992 | mask_palau_post1992].copy()
df_fossil_palau_fixed["Country"] = "PALAU"

# Step 6: Remove duplicate/conflicting 'PALAU' entries from main dataset
df_fossil_remaining = df_fossil_cleaned[
    ~(
        (df_fossil_cleaned["Country"] == "PACIFIC ISLANDS (PALAU)") |
        ((df_fossil_cleaned["Country"] == "PALAU") & (df_fossil_cleaned["Year"] < 1992))
    )
]

# Step 7: Merge cleaned 'PALAU' entries back
df_fossil_final = pd.concat(
    [df_fossil_remaining, df_fossil_palau_fixed],
    ignore_index=True
)

# Step 8: Sort and round numeric values
df_fossil_final = df_fossil_final.sort_values(by=["Country", "Year"]).reset_index(drop=True)
df_fossil_final = df_fossil_final.round(3)

# df_fossil_final now holds the fully cleaned fossil CO₂ data

In [5]:
# Rename dictionary for harmonizing country names
rename_dict = {
    "TÜRKIYE": "TURKEY",
    "CÔTE D'IVOIRE": "COTE D'IVOIRE",
    "BRUNEI DARUSSALAM": "BRUNEI",
    "DEMOCRATIC REPUBLIC OF THE CONGO": "DEMOCRATIC REPUBLIC OF CONGO",
    "STATE OF PALESTINE": "PALESTINE",
    "TIMOR-LESTE": "EAST TIMOR",
    "USA": "UNITED STATES",
    "VIET NAM": "VIETNAM",
    "BONAIRE, SAINT EUSTATIUS AND SABA": "BONAIRE SINT EUSTATIUS AND SABA",
    "CURAÇAO": "CURACAO",
    "FAEROE ISLANDS": "FAROE ISLANDS",
    "MICRONESIA (FEDERATED STATES OF)": "MICRONESIA (COUNTRY)",
    "WALLIS AND FUTUNA ISLANDS": "WALLIS AND FUTUNA",
    "BUNKERS": "INTERNATIONAL TRANSPORT"
}

# Region/aggregate names to drop
excluded_regions = {
    "AFRICA", "ASIA", "CENTRAL AMERICA", "EUROPE", "NORTH AMERICA", "OCEANIA",
    "SOUTH AMERICA", "EU27", "KP ANNEX B", "MIDDLE EAST", "NON KP ANNEX B",
    "NON-OECD", "OECD", "STATISTICAL DIFFERENCE"
}

# Function to load and clean a sheet
def load_and_rename_sheet(sheet_name):
    df_raw = pd.read_excel(source_path, sheet_name=sheet_name, skiprows=6)
    country_row = df_raw.iloc[1, 1:].astype(str).str.upper().str.strip()
    df_raw.iloc[1, 1:] = country_row.replace(rename_dict)
    return df_raw

# Function to process and merge consumption and trade data
def process_consumption_and_trade(df_consumption_raw, df_trade_raw):
    country_names = df_consumption_raw.iloc[1, 1:].astype(str).str.upper().str.strip().replace(rename_dict)
    columns = ["Year"] + country_names.tolist()

    df_consumption = df_consumption_raw.iloc[2:].reset_index(drop=True)
    df_trade = df_trade_raw.iloc[2:].reset_index(drop=True)
    df_consumption.columns = columns
    df_trade.columns = columns

    df_consumption_long = df_consumption.melt(id_vars="Year", var_name="Country", value_name="consumption_co2")
    df_trade_long = df_trade.melt(id_vars="Year", var_name="Country", value_name="trade_co2")

    df_merged = pd.merge(df_consumption_long, df_trade_long, on=["Year", "Country"])

    df_merged["consumption_co2"] = (df_merged["consumption_co2"].astype(float) * 3.664).round(3)
    df_merged["trade_co2"] = (-1 * df_merged["trade_co2"].astype(float) * 3.664).round(3)

    df_merged = df_merged.dropna(subset=["Country", "Year"])
    df_merged = df_merged[~df_merged["Country"].isin(excluded_regions)]

    return df_merged.sort_values(by=["Country", "Year"]).reset_index(drop=True)

# Load sheets
df_consumption_raw = load_and_rename_sheet("Consumption Emissions")
df_trade_raw = load_and_rename_sheet("Emissions Transfers")

# Process data
df_consumption_trade_final = process_consumption_and_trade(df_consumption_raw, df_trade_raw)

# df_consumption_trade_final is now ready in memory


In [6]:
# Step 1: Define file and sheet names
landuse_sheets = ["BLUE", "H&C2023", "OSCAR", "LUCE"]

# Step 2: Load all sheets into a dictionary
df_landuse_sheets = {
    sheet: pd.read_excel(landuse_file_path, sheet_name=sheet, header=None)
    for sheet in landuse_sheets
}

# Step 3: Extract GLOBAL column from each sheet
global_avg_dfs = []

for sheet_name, df_raw in df_landuse_sheets.items():
    header_row = df_raw.iloc[7, 1:].astype(str).str.upper().str.strip()
    df_raw.columns = [None] + list(header_row)
    
    df_years = df_raw.iloc[8:, 0]
    df_values = df_raw.iloc[8:, 1:]
    df_values.columns = header_row
    df_values.insert(0, "Year", df_years)

    if "GLOBAL" not in df_values.columns:
        raise ValueError(f"'GLOBAL' column not found in sheet: {sheet_name}")
    
    df_global = df_values[["Year", "GLOBAL"]].copy()
    df_global["GLOBAL"] = pd.to_numeric(df_global["GLOBAL"], errors="coerce")
    global_avg_dfs.append(df_global)

# Step 4: Merge all GLOBAL columns and compute row-wise mean
df_global_merged = global_avg_dfs[0]

for df_other in global_avg_dfs[1:]:
    df_global_merged = df_global_merged.merge(df_other, on="Year", suffixes=("", "_drop"))

df_global_merged["GLOBAL_AVG"] = df_global_merged.iloc[:, 1:].mean(axis=1)

# Step 5: Load and rename BLUE sheet headers
df_landuse_blue_raw = df_landuse_sheets["BLUE"]

rename_dict = {
    "TÜRKIYE": "TURKEY",
    "CÔTE D'IVOIRE": "COTE D'IVOIRE",
    "BRUNEI DARUSSALAM": "BRUNEI",
    "DEMOCRATIC REPUBLIC OF THE CONGO": "DEMOCRATIC REPUBLIC OF CONGO",
    "STATE OF PALESTINE": "PALESTINE",
    "TIMOR-LESTE": "EAST TIMOR",
    "USA": "UNITED STATES",
    "VIET NAM": "VIETNAM"
}

header_countries = df_landuse_blue_raw.iloc[7, 1:].astype(str).str.upper().str.strip()
header_countries = header_countries.replace(rename_dict)
df_landuse_blue_raw.iloc[7, 1:] = header_countries

# Step 6: Replace only the GLOBAL column with GLOBAL_AVG
if "GLOBAL" in header_countries.values:
    global_col_index = header_countries.tolist().index("GLOBAL") + 1  # Adjust for first column being year
    df_landuse_blue_raw.iloc[8:, global_col_index] = df_global_merged["GLOBAL_AVG"].values
else:
    raise ValueError("'GLOBAL' column not found in BLUE sheet after renaming.")

# Step 7: Convert BLUE sheet into long format for land-use emissions
country_names = df_landuse_blue_raw.iloc[7, 1:].astype(str).str.strip()
years = df_landuse_blue_raw.iloc[8:, 0]
df_landuse_values = df_landuse_blue_raw.iloc[8:, 1:]
df_landuse_values.columns = country_names
df_landuse_values.insert(0, "Year", years)

df_landuse_long_final = df_landuse_values.melt(
    id_vars="Year", var_name="Country", value_name="land_use_change_co2"
).dropna()

# Step 8: Scale and round values
df_landuse_long_final["land_use_change_co2"] = (
    df_landuse_long_final["land_use_change_co2"].astype(float) * 3.664
).round(3)

# Step 9: Sort and reset index
df_landuse_long_final = df_landuse_long_final.sort_values(["Country", "Year"]).reset_index(drop=True)

# Final cleaned land-use CO₂ emissions data is available in-memory:
# => df_landuse_long_final


In [7]:
# Step 1: Rename and clean fossil emissions data
df_fossil_renamed = df_fossil_final.rename(columns={
    "Country": "country",
    "Year": "year",
    "Country_Abbreviation": "Country_abbrievation",
    "Total": "co2",
    "Coal": "coal_co2",
    "Oil": "oil_co2",
    "Gas": "gas_co2",
    "Cement": "cement_co2",
    "Flaring": "flaring_co2",
    "Other": "other_industry_co2"
})

df_fossil_selected = df_fossil_renamed[[
    "country", "year", "Country_abbrievation", "co2", "cement_co2",
    "coal_co2", "flaring_co2", "gas_co2", "oil_co2", "other_industry_co2"
]]

# Step 2: Prepare land-use data
df_landuse_cleaned = df_landuse_long_final.rename(columns={"Country": "country", "Year": "year"})
df_landuse_cleaned["country"] = df_landuse_cleaned["country"].replace("GLOBAL", "WORLD")
df_landuse_selected = df_landuse_cleaned[["country", "year", "land_use_change_co2"]]

# Step 3: Prepare consumption + trade emissions data
df_consumption_trade_cleaned = df_consumption_trade_final.rename(columns={"Country": "country", "Year": "year"})
df_consumption_trade_selected = df_consumption_trade_cleaned[["country", "year", "consumption_co2", "trade_co2"]]

# Step 4: Merge all three datasets
df_emissions_merged = pd.merge(df_fossil_selected, df_landuse_selected, on=["country", "year"], how="outer")
df_emissions_final = pd.merge(df_emissions_merged, df_consumption_trade_selected, on=["country", "year"], how="outer")

# Step 5 Export the final dataframe to CSV
df_emissions_final.to_csv(OUTPUT_PATH, index=False)
print("Final dataset exported as 'Final_CO2_Emissions_Dataset.csv'")


Final dataset exported as 'Final_CO2_Emissions_Dataset.csv'
